In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.impute import KNNImputer
warnings.filterwarnings("ignore")
import re


In [45]:
df = pd.read_csv('train.csv')
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [46]:
df.columns

Index(['id', 'scrape_id', 'last_scraped', 'name', 'description', 'picture_url',
       'host_id', 'host_name', 'host_since', 'host_is_superhost',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates',
       'bathrooms_text', 'beds', 'amenities', 'price', 'minimum_nights',
       'maximum_nights', 'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'instant_bookable',
       'calculated_host_listings_count',
       'calculated_host_listi

In [47]:
import collections


In [48]:
def get_unique_words(df, column_name):
    x = collections.Counter([y for x in df.loc[:,column_name].astype(str).values.flatten() for y in x.lower().replace('[', '').replace(']', '').replace(' ',"").replace('\'',"").replace('"',"").split(",")])
    return x

In [49]:
unique_amenities = get_unique_words(df, 'amenities')

In [50]:
# unique_amenities.most_common(100)

In [51]:
def find_keyword(text, keyword):
    index = text.find(keyword)
    if index != -1 and index >= 2:
        return text[index - 2]
    else:
        return None


In [52]:
def find_keyword(text, keyword, location=-2, length=1):
    index = text.find(keyword)
    if index == -1:
        return None
    try:
        return text[index + location:index + location + length]
    except:
        return None


In [53]:

from sklearn.preprocessing import MinMaxScaler
def cleanData(df, test):

    ## Find Stars for Ratings
    df['ratings'] = df['name'].apply(lambda x: find_keyword(x, '\u2605', 1, 4))
    test['ratings'] = test['name'].apply(lambda x: find_keyword(x, '\u2605', 1, 4))

    # Create dummy columns for 'new' and 'none'
    df['new_rating'] = df['ratings'].apply(lambda x: 1 if x == 'New ' else 0)
    test['new_rating'] = test['ratings'].apply(lambda x: 1 if x == 'New ' else 0)

    # Convert continuous ratings to numeric, handling errors by setting them to NaN
    df['ratings'] = pd.to_numeric(df['ratings'], errors='coerce')
    test['ratings'] = pd.to_numeric(test['ratings'], errors='coerce')

    # Create quintiles for continuous ratings and one-hot encode them
    # Separate NaN and non-NaN values
    non_nan_values = df['ratings'].dropna()

    # Create quintiles for non-NaN values
    quintiles, cutoffs = pd.qcut(non_nan_values, q=3, labels=False, retbins=True, precision=1)
    df.loc[~df['ratings'].isna(), 'quintile'] = pd.cut(non_nan_values, bins=cutoffs, labels=False)
    test.loc[~test['ratings'].isna(), 'quintile'] = pd.cut(non_nan_values, bins=cutoffs, labels=False)

    # Handle NaN values (assign NaN to quintile column)
    df.loc[df['ratings'].isna(), 'quintile'] = None
    test.loc[test['ratings'].isna(), 'quintile'] = None

    # Convert quintile column to integer type
    df['quintile'] = df['quintile'].astype(pd.Int64Dtype())
    test['quintile'] = test['quintile'].astype(pd.Int64Dtype())

    # Generate dummy variables
    df = pd.get_dummies(df, columns=['quintile'], prefix='rating', dummy_na=True)
    df = df.drop("ratings", axis=1)
    test = pd.get_dummies(test, columns=['quintile'], prefix='rating', dummy_na=True)
    test = test.drop("ratings", axis=1)

    ## Change Date for Host Since
    latest = pd.to_datetime(df['host_since']).sort_values().iloc[-1]
    df["host_days"] = (latest - pd.to_datetime(df['host_since'])).dt.days
    df = df.drop("host_since", axis=1)
    latest = pd.to_datetime(test['host_since']).sort_values().iloc[-1]
    test["host_days"] = (latest - pd.to_datetime(test['host_since'])).dt.days
    test = test.drop("host_since", axis=1)

    ## Find Bathrooms
    df["num_bathrooms"] = df['bathrooms_text'].astype(str).str.extract(r'(\d+(\.\d+)?)')[0].astype(float)
    df = df.drop("bathrooms_text", axis=1)
    test["num_bathrooms"] = test['bathrooms_text'].astype(str).str.extract(r'(\d+(\.\d+)?)')[0].astype(float)
    test = test.drop("bathrooms_text", axis=1)

    ## Find Bedrooms
    df["num_bedrooms"] = df['name'].apply(lambda x: find_keyword(x, 'bedroom')).astype(float)
    df = df.drop("name", axis=1)
    test["num_bedrooms"] = test['name'].apply(lambda x: find_keyword(x, 'bedroom')).astype(float)
    test = test.drop("name", axis=1)

    ## Categorise Amenities
    amenities_list = get_unique_words(df, 'amenities').most_common(100)
    df['amenities'] = df['amenities'].astype('string')
    test['amenities'] = test['amenities'].astype('string')

    df['amenities'] = df['amenities'].apply(lambda x: x.lower().replace('[', '').replace(']', '').replace(' ', '').replace('"', ''))
    test['amenities'] = test['amenities'].apply(lambda x: x.lower().replace('[', '').replace(']', '').replace(' ', '').replace('"', ''))

    string_amenities = [amenity[0] for amenity in amenities_list if isinstance(amenity[0], str)]
    for amenity in string_amenities:
        df[amenity] = df['amenities'].apply(lambda x: 1 if amenity.lower() in x.lower() else 0)
        test[amenity] = test['amenities'].apply(lambda x: 1 if amenity.lower() in x.lower() else 0)
    df = df.drop("amenities", axis=1)
    test = test.drop("amenities", axis=1)

    # Categorise host verifications
    verif_list = ['email', 'phone', 'work_email']
    for verif in verif_list:
        df[verif] = df['host_verifications'].apply(lambda x: 1 if verif in x else 0)
        test[verif] = test['host_verifications'].apply(lambda x: 1 if verif in x else 0)
    df = df.drop("host_verifications", axis=1)
    test = test.drop("host_verifications", axis=1)

    # converting false and true to binary
    df.replace({'f': 0, 't': 1}, inplace=True)
    test.replace({'f': 0, 't': 1}, inplace=True)

    # dropping all useless variables
    to_drop = ["host_name", "host_id", 'scrape_id', 'last_scraped', 'description', 'picture_url','latitude',
       'longitude','calendar_last_scraped']
    df = df.drop(to_drop, axis=1)
    test = test.drop(to_drop, axis=1)

    ## dummify all categorical variables
    categorical_columns = df.select_dtypes(include=['object']).columns
    df = pd.get_dummies(df, columns=categorical_columns, prefix=categorical_columns, dummy_na=True)
    test = pd.get_dummies(test, columns=categorical_columns, prefix=categorical_columns, dummy_na=True)

    # Multiply the new columns with dummy variables
    property_type_df = df.filter(like="property_type")
    property_type_test = test.filter(like="property_type")
    df = pd.concat([df, (df["num_bedrooms"].to_numpy().reshape(-1, 1) * property_type_df).add_suffix('_bed')], axis=1)
    test = pd.concat([test, (test["num_bedrooms"].to_numpy().reshape(-1, 1) * property_type_test).add_suffix('_bed')], axis=1)
    df = pd.concat([df, (df["num_bathrooms"].to_numpy().reshape(-1, 1) * property_type_df).add_suffix('_bath')], axis=1)
    test = pd.concat([test, (test["num_bathrooms"].to_numpy().reshape(-1, 1) * property_type_test).add_suffix('_bath')], axis=1)

    # Multiply the new columns with dummy variables
    property_type_df = df.filter(like="room_type")
    property_type_test = test.filter(like="room_type")
    df = pd.concat([df, (df["num_bedrooms"].to_numpy().reshape(-1, 1) * property_type_df).add_suffix('_bed')], axis=1)
    test = pd.concat([test, (test["num_bedrooms"].to_numpy().reshape(-1, 1) * property_type_test).add_suffix('_bed')], axis=1)
    df = pd.concat([df, (df["num_bathrooms"].to_numpy().reshape(-1, 1) * property_type_df).add_suffix('_bath')], axis=1)
    test = pd.concat([test, (test["num_bathrooms"].to_numpy().reshape(-1, 1) * property_type_test).add_suffix('_bath')], axis=1)

    rating_type_df = df.filter(like="neighbourhood_cleansed")
    rating_type_test = test.filter(like="neighbourhood_cleansed")
    df = pd.concat([df, (df["availability_30"].to_numpy().reshape(-1, 1) * rating_type_df).add_suffix('_avail30')], axis=1)
    test = pd.concat([test, (test["availability_30"].to_numpy().reshape(-1, 1) * rating_type_test).add_suffix('_avail30')], axis=1)
    df = pd.concat([df, (df["availability_60"].to_numpy().reshape(-1, 1) * rating_type_df).add_suffix('_avail60')], axis=1)
    test = pd.concat([test, (test["availability_60"].to_numpy().reshape(-1, 1) * rating_type_test).add_suffix('_avail60')], axis=1)
    df = pd.concat([df, (df["availability_365"].to_numpy().reshape(-1, 1) * rating_type_df).add_suffix('_avail365')], axis=1)
    test = pd.concat([test, (test["availability_365"].to_numpy().reshape(-1, 1) * rating_type_test).add_suffix('_avail365')], axis=1)
    df = pd.concat([df, (df["availability_90"].to_numpy().reshape(-1, 1) * rating_type_df).add_suffix('_avail90')], axis=1)
    test = pd.concat([test, (test["availability_90"].to_numpy().reshape(-1, 1) * rating_type_test).add_suffix('_avail90')], axis=1)


    #adding bed * bath
    df['bed_bath_product'] = df['num_bathrooms'] * df['num_bedrooms']
    test['bed_bath_product'] = test['num_bathrooms'] * test['num_bedrooms']

    df.rename(columns={'rating_<NA>': 'none_rating'}, inplace=True)
    test.rename(columns={'rating_<NA>': 'none_rating'}, inplace=True)
    # Initialize KNNImputer with the desired number of neighbors (k)
    knn_imputer = KNNImputer(n_neighbors=5)

    # Impute missing values in the DataFrame
    imputed_data = knn_imputer.fit_transform(df)
    # Convert the imputed data back to a DataFrame
    df = pd.DataFrame(imputed_data, columns=df.columns)

    # Impute missing values in the DataFrame
    imputed_test = knn_imputer.fit_transform(test)
    # Convert the imputed data back to a DataFrame
    test = pd.DataFrame(imputed_test, columns=test.columns)

    df, test = df.align(test, axis=1, fill_value=0)
    
    # Identify numerical columns
    numerical_columns = df.select_dtypes(include=['number']).columns.difference(['price','id'])

    # Min-max scale numerical columns
    scaler = MinMaxScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    test[numerical_columns] = scaler.fit_transform(test[numerical_columns])
    test = test.drop("price", axis=1)
    return df, test

In [54]:
cleaned_train, cleaned_test = cleanData(train, test)

In [55]:
cleaned_train.to_csv("cleaned_train.csv", index=False)
cleaned_test.to_csv("cleaned_test.csv", index=False)

In [56]:
cleaned_train['id']

0            0.0
1            1.0
2            2.0
3            3.0
4            4.0
          ...   
15436    15436.0
15437    15437.0
15438    15438.0
15439    15439.0
15440    15440.0
Name: id, Length: 15441, dtype: float64

In [57]:
for col in cleaned_train.columns:
    print(col)

accommodates
airconditioning
availability_30
availability_365
availability_60
availability_90
backyard
bakingsheet
barbecueutensils
bathtub
bbqgrill
beachessentials
bed_bath_product
bedlinens
beds
blender
boardgames
bodysoap
booksandreadingmaterial
calculated_host_listings_count
calculated_host_listings_count_entire_homes
calculated_host_listings_count_private_rooms
calculated_host_listings_count_shared_rooms
carbonmonoxidealarm
ceilingfan
centralairconditioning
centralheating
children\u2019sdinnerware
cityskylineview
cleaningavailableduringstay
cleaningproducts
clothingstorage
clothingstorage:closet
coffee
coffeemaker
coffeemaker:dripcoffeemaker
coffeemaker:keurigcoffeemachine
conditioner
cookingbasics
crib
dedicatedworkspace
diningtable
dishesandsilverware
dishwasher
dryer
dryingrackforclothing
elevator
email
essentials
ethernetconnection
exerciseequipment
extrapillowsandblankets
fireextinguisher
firepit
firstaidkit
freedryer\u2013inunit
freeparkingonpremises
freestreetparking
freewa